# Clip indicators and make extent smaller for zonation subregion runs

This code was used to clip the indicators for each zonation subregion. This was necessary because the entire Southeast region is too large. When we were working in just the South Atlantic, we could input all the full indicator layers, and use a subregion mask to mask down. But for the full Southeast, this is too big and zonation can't run. This code is used to both mask the indicators and make the extent match the subregion extent, so that zonation can run.

It puts the outputs in the same directory as the inputs, but they will have names that start with A_ plus the subregion name so you can tell them apart. 

If an output has the same values as the input (usually ranging from 1 - 10ish), then there is overlap and the indicator should be used. If the output has a werid high max value (the value of NODATA in the input indicator, sometimes 65535 because some inputs were 16 bit unsigned) you know it doesn't overlap with this subregion.

In [8]:
import os, fnmatch
import arcpy

In [9]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
# folder that contains all the indicators
SourceWorkspace= r"C:\zonation\bin\indicators"

In [10]:
# define inputs
# this is the subregion that will be used as the extent for cell size, extent, and snapping
Subrgn= r"C:\zonation\bin\masks\Piedmont_RZ.tif"

# this is the name that will be put in front of the output
SubrgnN= "Piedmont_"

### Start Analysis

In [11]:
# Set the workspace where the indicators are
arcpy.env.workspace = SourceWorkspace

### Make a list of indicators to be clipped
Now that our extent is so large, we can no longer just input the full extent into our zonation subregion runs. We have to generate masks that have the extent for each subregion, and then we need to clip all the indicators for that zonation run to that smaller mask area as well. Blah.

In [12]:
# make a list of indicators to be clipped
RList = arcpy.ListRasters() 

In [13]:
# print list of indicators to make sure it looks right
print(RList)

['IndicatorBins2019.tif']


### loop through indicators and clip to subregion

In [14]:
# Loop through all indicator rasters, clip to subregion and change extent
for r in RList:
    # wrap this in a try/except clause to deal with huge numbers as noData in some indicators 
    # These indicators only have noData values in this subregion but the code converts them 
    # to the numerical noData values which causes a runtime error if they're too large
    try:
        # Process: 
        with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Subrgn, snapRaster=Subrgn, cellSize=Subrgn):
            out_raster = arcpy.sa.ExtractByMask(r, Subrgn); out_raster.save(SubrgnN + r)
            # it doesn't re-calculate statistic after the mask, so it still shows the old full attribute
            # table, but adding statistics here also didn't fix the attribute table problem
            #arcpy.management.CalculateStatistics(SubrgnN + r, 1, 1, [], "OVERWRITE", r"in_memory\feature_set1")
            # I ended up using ye old *1 trick to force statistics calculation, so we can tell
            # which indicators overlap and which do not
            out_raster = arcpy.sa.Times(SubrgnN + r, 1); out_raster.save('A_' + SubrgnN + r)
            # this just does a bit of clean up and deletes the output with the old full attribute table
            arcpy.management.Delete(SubrgnN + r, '')
            arcpy.management.CalculateStatistics('A_' + SubrgnN + r, 1, 1, [], "OVERWRITE", r"in_memory\feature_set1")
    except RuntimeError:
        print(r+" is just noData and noData value is too large. Skipping this indicator")
        
print ("finished clipping indicators") 

finished clipping indicators
